In [1]:
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
import cv2
import numpy as np
import time

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300, fps=5)
robot = Robot()

In [2]:
###### definitions #######

target = 1 # arbitrary final target (human)
obstacle = 3 # arbitrary obstacle (apple)
global backingup
backingup = False
isEndReached = False


In [3]:
imageFeed = widgets.Image(format='jpeg', width=300, height=300)
speedSlider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, description='speed')

width = int(imageFeed.width)
height = int(imageFeed.height)

In [14]:
def move_back(direction="left"):
    robot.stop()
    robot.backward(speedSlider.value)

    global backingup
    backingup = True
    
def avoid_obstacle(direction="left"):
    robot.left(speedSlider.value)
    time.sleep(0.4)
    robot.stop()
    
    robot.left_motor.value = speedSlider.value + 0.1
    robot.right_motor.value = speedSlider.value
    time.sleep(1.5)
    robot.stop()


In [5]:
display(widgets.VBox([widgets.HBox([imageFeed]), speedSlider]))

object_arr = [1, 3] # List of valid obstacles
diff = 50

def run(change):
    image = change['new']
    
    detections = model(image)
    imageFeed.value = bgr8_to_jpeg(image)
    
    for detection in detections[0]:
        bbox = detection['bbox']
        label = detection['label']
        
        if label in object_arr:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (92, 212, 74), 2)
            cv2.putText(image, "", (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (92, 212, 74))
            
            if int(height * bbox[3]) > height - diff:
                move_back()
                
            global backingup
            if backingup and int(height * bbox[3]) < height - diff + 10:
                robot.stop()
                backingup = False
                avoid_obstacle()
    
    imageFeed.value = bgr8_to_jpeg(image)
    
    # additional methds, although the lock on method will probably just be added to original file

In [26]:
camera.observe(run, names='value')  

In [27]:
robot.forward(speedSlider.value)

In [25]:
camera.unobserve_all()
robot.stop()